In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from math import sqrt
import itertools
import seaborn as sns

# Função para calcular distância de Hellinger
def hellinger(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    return (1 / sqrt(2)) * np.linalg.norm(np.sqrt(p) - np.sqrt(q))

# Simular 6 investidores, cada um com 5 distribuições (temporal, geográfica, setorial, estágio, montante)
np.random.seed(42)
num_investidores = 6
num_bins = 10
num_metricas = 5

# Criar distribuições empíricas normalizadas para cada investidor
investidores = {
    f"I{i}": [np.random.dirichlet(np.ones(num_bins)) for _ in range(num_metricas)]
    for i in range(num_investidores)
}

# Calcular matriz de similaridade (distâncias de Hellinger médias)
dist_matrix = np.zeros((num_investidores, num_investidores))

for i, j in itertools.combinations(range(num_investidores), 2):
    inv_i = list(investidores.values())[i]
    inv_j = list(investidores.values())[j]
    dist = np.mean([hellinger(inv_i[m], inv_j[m]) for m in range(num_metricas)])
    dist_matrix[i, j] = dist
    dist_matrix[j, i] = dist

# Criar grafo a partir de uma matriz de similaridade com threshold
threshold = 0.3  # quanto menor, mais exigente
G = nx.Graph()

# Adicionar nós
for name in investidores.keys():
    G.add_node(name)

# Adicionar arestas com base na similaridade
investidor_nomes = list(investidores.keys())
for i in range(num_investidores):
    for j in range(i + 1, num_investidores):
        if dist_matrix[i, j] < threshold:
            G.add_edge(investidor_nomes[i], investidor_nomes[j], weight=1 - dist_matrix[i, j])

# Detectar comunidades com algoritmo de Louvain (usando greedy modularity como alternativa)
from networkx.algorithms.community import greedy_modularity_communities
comunidades = list(greedy_modularity_communities(G))

# Preparar para visualização
color_map = {}
for c_idx, community in enumerate(comunidades):
    for node in community:
        color_map[node] = c_idx
colors = [color_map[node] for node in G.nodes]

# Plotar o grafo com cores por comunidade
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color=colors, node_size=700, cmap=plt.cm.Set1)
plt.title("Clusters de investidores detectados via comunidades")
plt.axis('off')
plt.tight_layout()
plt.show()

# Mostrar a matriz de distâncias como heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(dist_matrix, xticklabels=investidor_nomes, yticklabels=investidor_nomes, cmap="Reds", annot=True)
plt.title("Matriz de distância média entre investidores")
plt.tight_layout()
plt.show()

